In [189]:
import random
import math
import numpy as np

random.seed(222)

import gurobipy as gp
from gurobipy import GRB

number_of_items=500 
max_cost=10
max_weight=13
rhs_p=0.5

#last item 
use_last_item=0
last_item=10000

#compare variables IP vs. LP
compar_vars=0

c=list()
w=list()
rhs=0

for i in range(number_of_items):
    c.append(random.randint(1,max_cost)) 
    w.append(random.randint(1,max_weight-c[i]))
    rhs+=w[i]

rhs=math.floor(rhs_p*rhs)

if(use_last_item==1):
    c[number_of_items-1]=last_item
    w[number_of_items-1]=last_item-1
    rhs=last_item-1


# Create IP 
m = gp.Model()
x = m.addVars(number_of_items,vtype=GRB.BINARY)
m.setObjective(gp.quicksum(c[i]*x[i] for i in range(number_of_items)), GRB.MAXIMIZE )
m.addConstr(gp.quicksum(w[i]*x[i] for i in range(number_of_items)) <= rhs)

m.write("KnapsackIP.lp")
m.optimize()

vars = m.getVars()

objIP=m.getObjective()
print("optimal IP objective: ",objIP.getValue())

if(use_last_item==1):
    print("last item variable: ",vars[number_of_items-1].X)
    sum_of_other_variables=0
    for i in range(number_of_items-1):
        sum_of_other_variables+=vars[i].X
    print("sum_of_other_variables: ",sum_of_other_variables)
    


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1 rows, 500 columns and 500 nonzeros
Model fingerprint: 0xbb769fab
Variable types: 0 continuous, 500 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 1489.0000000
Presolve removed 0 rows and 425 columns
Presolve time: 0.00s
Presolved: 1 rows, 75 columns, 75 nonzeros
Variable types: 0 continuous, 75 integer (1 binary)
Found heuristic solution: objective 1712.0000000

Root relaxation: objective 2.401750e+03, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

In [190]:
# Create LP 
mLP = gp.Model()
xLP = mLP.addVars(number_of_items,lb=0,ub=1,vtype=GRB.CONTINUOUS)
mLP.setObjective(gp.quicksum(c[i]*xLP[i] for i in range(number_of_items)), GRB.MAXIMIZE )
mLP.addConstr(gp.quicksum(w[i]*xLP[i] for i in range(number_of_items)) <= rhs)

mLP.write("KnapsackLP.lp")
mLP.optimize()

objLP=mLP.getObjective()

print("optimal LP objective: ",objLP.getValue())
print("ratio of objective LP/IP: ", objLP.getValue()/objIP.getValue())

varsLP = mLP.getVars()

if(use_last_item==1):
    print("last item variable: ",varsLP[number_of_items-1].X)
    sum_of_other_variables=0
    for i in range(number_of_items-1):
        sum_of_other_variables+=varsLP[i].X
    print("sum_of_other_variables: ",sum_of_other_variables)

if(compar_vars==1):
       for i in range(number_of_items):
            if(abs(vars[i].X-varsLP[i].X)>0.0001):
                print("index: ", i," IP: ",vars[i].X," LP: ",varsLP[i].X)
#                print("c: ", c[i]," w: ",w[i])
                


    

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1 rows, 500 columns and 500 nonzeros
Model fingerprint: 0x925bafa5
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Presolve removed 0 rows and 446 columns
Presolve time: 0.00s
Presolved: 1 rows, 54 columns, 54 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0820000e+04   6.650000e+01   0.000000e+00      0s
       1    2.4017500e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.401750000e+03
optimal LP objective:  2401.75
ratio of objective LP/IP:  1.0003123698458976


In [179]:
x=5+3/4-2-3
x

0.75